In [1]:
import os
from langchain_openai.embeddings import OpenAIEmbeddings
from openai import OpenAI
from langchain_chroma.vectorstores import Chroma
from langchain_community.document_loaders import Docx2txtLoader
from glob import glob
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker

In [2]:
from dotenv import load_dotenv
load_dotenv()
HF_TOKEN = os.getenv('HF_TOKEN')

In [3]:
client = OpenAI()
emb = OpenAIEmbeddings( model="text-embedding-3-small" )

In [4]:
semantic_splitter = SemanticChunker(
    emb,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=50,
)
fallback_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400, chunk_overlap=80, separators=["\n\n", "\n", " ", ""]
)

In [5]:
# Docx file 전처리

docx_files = glob('./loan_data/*.docx')
documents = []

In [6]:
for docx_file in docx_files:
    loader = Docx2txtLoader(docx_file)
    docs = loader.load()
    print(docs)

[Document(metadata={'source': './loan_data\\1.청년전용_보증부월세대출.docx'}, page_content='[출처] 주택도시기금 청년 전월세보증금 대출\n\n[링크] https://nhuf.molit.go.kr/FP/FP05/FP0502/FP05020701.jsp\n\n[상품명] 청년전용 보증부월세대출\n\n주택도시기금의 개인상품 중 청년전용 보증부월세대출입니다.\n\n청년들에게 저리로 전월세보증금 및 월세를 대출해드립니다.\n\n[대출대상] 부부합산 연소득 5천만원 이하, 순자산가액 3.37억원 이하 무주택 단독 세대주(예비세대주 포함)\n만 19세 이상 ~ 만 34세 이하 청년\n\n[대출금리] (보증금) 연 1.3% (월세금) 연0% (20만원 한도),\xa0 1.0% (20만원 초과)\xa0\n\n[대출한도] (보증금) 최대 4,500만원 이내 (월세금) 최대 1,200만원 (24개월 기준: 월 50만원 이내)\xa0\n\n[대출기간] 25개월(최장 10년 5개월 이용가능)\n\n[대출 안내]\n\n1. 대출 대상\n\n아래의 요건을 모두 충족하는 자\n\n1. (계약) 주택임대차계약을 체결하고 임차보증금의 5% 이상을 지불한 자\n\n2. (세대주) 대출접수일 현재 민법상 성년인 단독세대주(예비세대주 포함)\n\n3. (무주택) 세대주를 포함한 세대원 전원이 무주택인 자\n\n4. (중복대출 금지) 주택도시기금대출, 은행재원 전세자금대출 및 주택담보대출 미이용자\n\n(주택도시기금대출) 성년인 세대원 전원(세대가 분리된 배우자 및 자녀, 결혼예정 배우자, 배우자의 직계존속과 동거세대를 구성하는 경우 배우자의 직계존속 포함)이 기금 대출을 이용 중이면 대출 불가\n\n(전세자금대출 및 주택담보대출) 차주 및 배우자(결혼예정 또는 분리된 배우자 포함)가 전세자금대출 및 주택담보대출을 이용 중이면 대출 불가\n\n(임차중도금대출 중복예외허용) 한국주택금융공사 주택보증서 담보로 취급된 기금 임차중도금(잔금포함) 대출을 이

In [7]:
for docx_file in docx_files:
    loader = Docx2txtLoader(docx_file)
    docs = loader.load()  # 각 페이지가 하나의 Document(메타데이터에 source/page 포함)

    sem_chunks = semantic_splitter.split_documents(docs)
    chunks = []
    for d in sem_chunks:
        if len(d.page_content) > 800:
            chunks.extend(fallback_splitter.split_documents([d]))
        else:
            chunks.append(d)

    documents.extend(chunks)

In [8]:
PERSIST_DIR = "./db" # 경로 혼동 방지(절대경로 추천)
vector_store_youth = Chroma(
    collection_name="YOUTH",     # 처음과 동일
    persist_directory=PERSIST_DIR,  # 처음과 동일
    embedding_function=emb,         # 동일 모델/차원
)

In [9]:
vector_store_marry1 = Chroma(
    collection_name="MARRY1",     # 처음과 동일
    persist_directory=PERSIST_DIR,  # 처음과 동일
    embedding_function=emb,         # 동일 모델/차원
)

In [10]:
vector_store_marry2 = Chroma(
    collection_name="MARRY2",     # 처음과 동일
    persist_directory=PERSIST_DIR,  # 처음과 동일
    embedding_function=emb,         # 동일 모델/차원
)

In [11]:
vector_store_youth.add_documents(documents)

['f25a3848-d53d-4677-8471-749dc13ff91f',
 'a0abe699-b176-441e-b049-38c74bd4d0f6',
 'f174b700-37c5-429b-a3c9-6ddffa54b0bf',
 '33ed2790-2b34-4aa1-a814-8505d5353a68',
 '5b663ae4-d38a-4389-a1b4-de86b5ad5d9c',
 '2b9a1dc4-4041-4c30-bd30-aefec014a179',
 '7ec3b7c0-bb32-4744-9bc5-ddd74f292286',
 'd3554c87-6ed6-4dd4-929e-ac53c5a4f094',
 '97e67909-ed1c-4fa9-91b7-f2d5450f7fa5',
 '5f86ab57-4b89-49a6-a0af-02dd2178b559',
 '3e9c7527-66c1-4808-9ad3-ef90b286a9a7',
 'dc3f6a80-1860-4982-89c1-687b291db6af',
 'c96bb4b3-f758-4aa6-bc1d-bbf6110fe1f4',
 '8e0e09ac-83a9-4522-abf8-4a3c452c3615',
 '871a5c1e-a4dc-41f3-8ed7-b9e0e623722f',
 'db628507-8363-4232-85a6-2765f4a92323',
 'a3bc758d-859a-4e22-b5c9-3a64098301c3',
 'c1d77945-d1e6-496b-bccf-9bf82aeaceab',
 'b386dd40-7a79-4a54-b6c4-15a6ab78ec5b',
 'e1dd5447-0871-4c07-b9b1-9c1ed26b5a38',
 'a688f976-8918-455f-b05c-d6d62eef79ad',
 'c8bf51e2-8bc9-4798-a81b-3258492d6740',
 '9a1a941c-cb67-4f13-979d-5c701e2310d4',
 'ba7675ae-21ce-48bb-a34e-9516519bb0b0',
 '72a696d5-2b99-

In [12]:
vector_store_marry1.add_documents(documents)

['dc8f714e-333d-413f-969c-160752a81d33',
 'e32a6c8e-2fcd-4c0d-8d7a-08e293b58386',
 '9db5dcc9-6b0a-42bc-8847-8230c1e1fd18',
 'e03f64ca-3984-4066-944c-f58e3a72d91e',
 '35da975e-6b88-4474-875c-bfccc54f8d27',
 'aa23e9f5-d7c1-4cbe-8971-d487ab9a2378',
 '3331410a-43a4-4311-92d0-88be0e2e0c05',
 '4b4ee84f-d062-453b-837f-33b06d675b30',
 '9a9b4e71-c90e-461e-b0b2-23dfe7bfb73b',
 '67687171-3547-4595-a786-837408193d8b',
 '620fd6dd-9c29-4414-9ee5-8443aa183b0d',
 'd00c9ff5-9afb-450d-ba8b-48ac33cb4540',
 '53a84eeb-13af-471b-be33-698eb7047bd7',
 '5f51cba9-48ae-429a-9120-cb1a9c74a4a4',
 'c8f4e642-28c2-4776-84d9-f6c2c4ba10a4',
 '7fb10a8c-6834-4435-b0ae-3791c3b80da2',
 '2e205018-7052-4fcc-87ab-93dd4535598e',
 '5d32e8ce-4308-4e89-a605-f056a12897fc',
 '63e76170-08ec-4efb-b5b8-2e48632e9c26',
 'd2149d18-d267-4671-9146-fa522a7db5e4',
 '53192778-d209-4007-9345-d3ce5cae2343',
 '8500bcb6-85b3-49dd-a8ca-3b1a83f25adc',
 '85ac0c56-eaed-40b4-885e-c4aad9855ada',
 '75a5c320-9cfc-41e8-9731-30d9ce67b6cc',
 '436dde26-6477-

In [13]:
vector_store_marry2.add_documents(documents)

['516ab407-3ec6-46bb-94bd-cc24a58c3860',
 '31078cc7-6f3d-4fc7-b4bc-d962bae33435',
 'b976d8f9-d8e9-43eb-ad11-a0ff82d28edb',
 '7b329041-b7b4-4401-ad0d-c04f5cbf02d7',
 'afb46849-1d33-43a7-bb13-7bbc393ced7c',
 '6938e1b1-8f05-465f-9a6a-a23d693dbf0f',
 '57f63264-070a-46e5-a7dc-2c9160d59c7d',
 '9fe77ce0-7dfb-4b15-82d9-642a1af9d439',
 '964f7a74-fbab-4486-968e-f5f482344449',
 'b733f5c5-212e-457b-9b2d-6eb2fdd9ad14',
 '570deea9-8d95-4341-bc41-e4a73d2aee41',
 '40800414-7a41-465f-b67f-c3e822185246',
 '0ab4ba76-3dea-4409-8b64-9847fd071011',
 '2b4c7033-e468-493e-8b08-71699edf3e66',
 '3aef740d-efc2-464f-927d-90430d761ee4',
 'a7fc73b2-042d-43f7-8e58-3d2f889f3967',
 'c27c5443-aa94-4495-b07f-a51caead6da7',
 '1ae2f073-f9ae-428d-b2a6-1063b8b46cf1',
 '359eb891-37d1-4461-b20a-016c36fa3c68',
 'df5be736-862b-4ed8-b1d7-333be4c9178e',
 '8e3f584a-7e6f-4d24-9384-1c09a2106818',
 '7441e94a-3df2-4ce9-bc7d-e689add702c2',
 'c58b15e6-bd21-4cec-9587-b2adc575252e',
 '19a1294d-c3c6-4d55-983c-287a595819b5',
 'a8b18929-5012-